In [1]:
import pandas as pd
import numpy as np
import math
import os

In [2]:
merged7 = pd.read_csv("baseregressions_inputs.csv")

In [3]:
merged7.columns

Index(['country', '('rgdpna', 1950)', '('rgdpna', 1951)', '('rgdpna', 1952)',
       '('rgdpna', 1953)', '('rgdpna', 1954)', '('rgdpna', 1955)',
       '('rgdpna', 1956)', '('rgdpna', 1957)', '('rgdpna', 1958)',
       ...
       '2008inflation', '2009inflation', '2010inflation', '2011inflation',
       '2012inflation', '2013inflation', '2014inflation', '2015inflation',
       '2016inflation', '2017inflation'],
      dtype='object', length=420)

In [4]:
merged7

,country,"('rgdpna', 1950)","('rgdpna', 1951)","('rgdpna', 1952)","('rgdpna', 1953)","('rgdpna', 1954)","('rgdpna', 1955)","('rgdpna', 1956)","('rgdpna', 1957)","('rgdpna', 1958)",...,2008inflation,2009inflation,2010inflation,2011inflation,2012inflation,2013inflation,2014inflation,2015inflation,2016inflation,2017inflation
0,Argentina,153269.50000,162321.85940,152653.73440,159095.50000,164275.73440,175750.10940,180385.10940,187317.2500,199688.56250,...,8.584014,6.282774,10.780115,9.465686,10.030259,10.619433,NaN,NaN,NaN,NaN
1,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.949953,3.406767,8.181576,7.651234,2.555471,5.793788,2.980988,3.726586,-1.271210,NaN
2,Australia,119859.33590,122907.87500,117876.97660,130664.96880,141111.00000,146676.98440,147013.64060,150233.7188,160424.40630,...,4.352643,1.820112,2.845226,3.303850,1.762780,2.449889,2.487923,1.508367,1.276991,NaN
3,Austria,46511.69531,49393.62891,49201.43359,50817.83984,55011.61328,61310.65625,65090.07813,68869.5000,71389.11719,...,3.215921,0.506313,1.813535,3.266939,2.485675,2.000157,1.605806,0.896567,0.891591,NaN
4,Bahrain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.526003,2.795513,1.961885,-0.364448,2.754554,3.305485,2.651195,1.835994,2.798520,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Turkey,69329.51563,85676.57813,92840.96094,104853.78130,94846.32813,104528.00780,105686.67970,128691.6797,138803.56250,...,10.444128,6.250977,8.566444,6.471880,8.891570,7.493090,8.854573,7.670854,7.775134,NaN
90,Ukraine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.231911,15.894569,9.378589,7.960095,0.555556,-0.276243,12.188366,48.724280,13.895038,NaN
91,United Kingdom,522552.59380,544855.18750,549122.31250,574452.50000,591526.81250,608731.93750,614828.62500,622461.2500,624467.06250,...,3.613499,2.166231,3.285714,4.484240,2.821710,2.554547,1.460192,0.050021,0.641613,NaN
92,Uruguay,14395.21973,16629.11914,16008.74121,16847.71875,18361.15234,18699.21875,18742.65430,19366.0332,18228.07813,...,7.860952,7.099167,6.676387,8.093775,8.097766,8.575135,8.877353,8.666270,9.639413,NaN


In [5]:
#demographic instrument: use male population ratio as instrument?
maleyoung = pd.read_csv("male population ratio.csv")
maleyoung = maleyoung.rename(columns= lambda x: x + 'maleyoung')
maleyoung = maleyoung.rename(columns = {'Country ' + 'maleyoung': 'country'})
maleyoung = maleyoung[['country','2000maleyoung', '2005maleyoung', '2010maleyoung']]

femaleyoung = pd.read_csv("female population ratio.csv")
femaleyoung = femaleyoung.rename(columns= lambda x: x + 'femaleyoung')
femaleyoung = femaleyoung.rename(columns = {'Country' + 'femaleyoung': 'country'})
femaleyoung = femaleyoung[['country','2000femaleyoung', '2005femaleyoung', '2010femaleyoung']]

totalyoung = pd.read_csv("total population ratio.csv")
totalyoung = totalyoung.rename(columns= lambda x: x + 'totalyoung')
totalyoung = totalyoung.rename(columns = {'Country' + 'totalyoung': 'country'})
totalyoung = totalyoung[['country','2000totalyoung', '2005totalyoung', '2010totalyoung']]

childrensexratio = maleyoung[['2000maleyoung', '2005maleyoung', '2010maleyoung']].to_numpy()/femaleyoung[['2000femaleyoung', '2005femaleyoung', '2010femaleyoung']].to_numpy()
childrensexratio = pd.DataFrame(childrensexratio, columns = ['2000sexratio', '2005sexratio', '2010sexratio'])
childrensexratio['country'] = maleyoung['country']

merged8 = pd.merge(merged7, childrensexratio, on ='country', how  = 'left')
merged8 = pd.merge(merged8, femaleyoung, on ='country', how  = 'left')
merged8 = pd.merge(merged8, maleyoung, on ='country', how  = 'left')
merged8 = pd.merge(merged8, totalyoung, on ='country', how  = 'left')




merged7 = merged8

In [6]:
GDP = merged8[["('rgdpna', 2000)", "('rgdpna', 2005)", "('rgdpna', 2010)", "('rgdpna', 2014)"]]
TFP = merged8[["('rtfpna', 2000)", "('rtfpna', 2005)", "('rtfpna', 2010)", "('rtfpna', 2014)"]]

In [7]:
GDPgrowth = pd.DataFrame(GDP.pct_change(axis='columns').iloc[:, 1:].values, columns = ['gdpgrowth2000', 'gdpgrowth2005', 'gdpgrowth2010'])

TFPgrowth = pd.DataFrame(TFP.pct_change(axis='columns').iloc[:, 1:].values, columns = ['tfpgrowth2000', 'tfpgrowth2005', 'tfpgrowth2010'])

paneldata = pd.concat([GDPgrowth, TFPgrowth], axis = 1)

In [8]:
paneldata

,gdpgrowth2000,gdpgrowth2005,gdpgrowth2010,tfpgrowth2000,tfpgrowth2005,tfpgrowth2010
0,0.104004,0.320711,0.129353,0.010692,0.105593,-0.014859
1,0.808074,0.208409,0.200263,0.900307,0.006927,0.144122
2,0.185403,0.142043,0.120231,0.065458,-0.001489,0.003269
3,0.088989,0.066370,0.042867,0.011002,-0.000589,-0.008203
4,0.286057,0.310533,0.164899,0.032611,-0.158898,0.044805
...,...,...,...,...,...,...
89,0.249368,0.170011,0.191171,0.096580,-0.046420,-0.021970
90,0.446740,0.051385,-0.015625,0.450243,0.094592,0.006738
91,0.148855,0.019776,0.085023,0.069728,-0.028972,0.014602
92,0.009168,0.335807,0.181938,-0.001198,0.171046,0.057874


In [9]:
logGDP = np.log(merged8[["('rgdpna', 2000)", "('rgdpna', 2005)", "('rgdpna', 2010)"]])
logTFP = np.log(merged8[["('rtfpna', 2000)", "('rtfpna', 2005)", "('rtfpna', 2010)"]])
logGDP.columns = ['gdp2000', 'gdp2005', 'gdp2010']
logTFP.columns = ['tfp2000', 'tfp2005', 'tfp2010']

In [10]:
yearsofschooling = merged8[[
                           '2000sexratio', '2005sexratio','2010sexratio', 
                           '2000maleyoung', '2005maleyoung', '2010maleyoung',
                           '2000femaleyoung', '2005femaleyoung', '2010femaleyoung',
                            '2000totalyoung', '2005totalyoung', '2010totalyoung']]
                            
yearsofschooling.columns = [
                            'sexratio2000', 'sexratio2005', 'sexratio2010',
                            'maleyoung2000', 'maleyoung2005', 'maleyoung2010',
                           'femaleyoung2000', 'femaleyoung2005', 'femaleyoung2010',
                              'totalyoung2000', 'totalyoung2005', 'totalyoung2010' ]                                                     


In [11]:
#fertility rate
# create a list of fertility rate column names
# 注意data periods 的end
start = 2000
end = 2014

fertilityc = []
k = start - 1
for i in range(end-start+1):
    k = k + 1
    name = str(k) + 'fertility rate'
    fertilityc.append(name)
print(fertilityc)

import math
def logg(x):
    return math.log(x)

#6
# Create fertility rate variable as average of fertility rates over period 
# create a list of fertility rate column names


def calaveferrate1(k, fertilitycolumnnames):
    if k != 10:
        avg = (merged7[fertilitycolumnnames[k]] + merged7[fertilitycolumnnames[k+1]] 
        + merged7[fertilitycolumnnames[k+2]] + merged7[fertilitycolumnnames[k+3]]
        + merged7[fertilitycolumnnames[k+4]])/5
    else:
        avg = (merged7[fertilitycolumnnames[k]] + merged7[fertilitycolumnnames[k+1]] 
        + merged7[fertilitycolumnnames[k+2]] + merged7[fertilitycolumnnames[k+3]])/4
    return avg



fertilityrate1 =  calaveferrate1(0,fertilityc)
fertilityrate2 =  calaveferrate1(5, fertilityc)
fertilityrate3 =  calaveferrate1(10,  fertilityc)

fertilityrate = pd.concat([fertilityrate1, fertilityrate2, fertilityrate3 ], axis = 1)
'''
for i in range(N-1):
    fertilityrate = fertilityrate.append(calaveferrate((i+1)*10, gdpgrowth.loc[str(i+2)], str(i+2),fertilityc))
'''
logfertilityrate = np.log(fertilityrate)

logfertilityrate.columns = ['logfertilityrate2000', 'logfertilityrate2005', 'logfertilityrate2010']
logfertilityrate

['2000fertility rate', '2001fertility rate', '2002fertility rate', '2003fertility rate', '2004fertility rate', '2005fertility rate', '2006fertility rate', '2007fertility rate', '2008fertility rate', '2009fertility rate', '2010fertility rate', '2011fertility rate', '2012fertility rate', '2013fertility rate', '2014fertility rate']


,logfertilityrate2000,logfertilityrate2005,logfertilityrate2010
0,0.922909,0.881699,0.855479
1,0.498713,0.528980,0.513572
2,0.561557,0.655341,0.646449
3,0.319181,0.337900,0.362905
4,0.977800,0.835861,0.756005
...,...,...,...
89,0.876884,0.796786,0.756826
90,0.132956,0.307338,0.395246
91,0.512824,0.614104,0.639219
92,0.791362,0.754242,0.720519


In [12]:
#9
# average of life expectancy over period 
# create a list of life expectancy column names
lifec = []
k = start-1
for i in range(end-start+1):
    k = k + 1
    name = str(k) + 'life expectancy'
    lifec.append(name)

lifeexpectancy1 =  calaveferrate1(0, lifec)
lifeexpectancy2 =  calaveferrate1(5, lifec)
lifeexpectancy3 =  calaveferrate1(10,  lifec)


lifeexpectancy = pd.concat([lifeexpectancy1, lifeexpectancy2, lifeexpectancy3], axis = 1)
loglifeexpectancy = np.log(lifeexpectancy)
loglifeexpectancy.columns = ['loglifeexpectancy2000', 'loglifeexpectancy2005', 'loglifeexpectancy2010']
loglifeexpectancy

,loglifeexpectancy2000,loglifeexpectancy2005,loglifeexpectancy2010
0,4.306041,4.317588,4.327228
1,4.278934,4.300386,4.309398
2,4.380861,4.397198,4.406065
3,4.364862,4.382417,4.393327
4,4.316758,4.326862,4.335161
...,...,...,...
89,4.263359,4.292533,4.310785
90,4.221912,4.224490,4.259782
91,4.359451,4.375499,4.392173
92,4.318625,4.330027,4.338794


In [13]:
#11
# Average of government consumption ratio
# create a list of government consumption column names
govc = []
k = start-1
for i in range(51):
    k = k + 1
    name = str(k) + 'government consumption'
    govc.append(name)
    
'''
gcratio =  calaveferrate(0, gdpgrowth.loc['1'], str(1), govc)
for i in range(N-1):
    gcratio = gcratio.append(calaveferrate((i+1)*5, gdpgrowth.loc[str(i+2)], str(i+2), govc))
'''
gcratio1 =  calaveferrate1(0, govc)
gcratio2 =  calaveferrate1(5,  govc)
gcratio3 =  calaveferrate1(10,  govc)

gcratio = pd.concat([gcratio1, gcratio2, gcratio3], axis = 1)
gcratio.columns = ['gcratio2000', 'gcratio2005', 'gcratio2010']

gcratio

,gcratio2000,gcratio2005,gcratio2010
0,12.545707,13.419602,16.076088
1,10.692434,10.876053,12.210910
2,17.598348,17.359175,17.978956
3,18.825773,19.359046,19.928484
4,16.844609,12.783381,14.357545
...,...,...,...
89,12.560573,13.658968,14.257984
90,19.205443,18.505290,19.204538
91,17.971147,20.122498,20.805139
92,12.037998,11.771557,13.050655


In [14]:
#12
# Average of inflation rate
# create a list of inflation rate names
inf = []
k = start-1
for i in range(51):
    k = k + 1
    name = str(k) + 'inflation'
    inf.append(name)
'''    
inflation =  calaveferrate(0, gdpgrowth.loc['1'], str(1), inf)
for i in range(N-1):
    inflation = inflation.append(calaveferrate((i+1)*5, gdpgrowth.loc[str(i+2)], str(i+2), inf))
'''

inflation1 =  calaveferrate1(0,  inf)
inflation2 =  calaveferrate1(5, inf)
inflation3 =  calaveferrate1(10,  inf)


inflation = pd.concat([inflation1, inflation2, inflation3], axis = 1)
inflation.columns = ['inflation2000', 'inflation2005', 'inflation2010']
inflation

,inflation2000,inflation2005,inflation2010
0,8.344898,8.847745,10.223873
1,3.019577,4.059756,6.045517
2,3.394709,2.942467,2.590436
3,2.055500,1.929043,2.391577
4,0.307720,2.834372,1.914369
...,...,...,...
89,38.032059,9.037387,7.855746
90,11.029414,15.318514,4.404499
91,1.196975,2.496792,3.286553
92,10.326595,6.834503,7.860766


In [15]:
#13
#Rule of law & voice

indexes =  merged8[['2000ruleoflaw', '2005ruleoflaw', '2010ruleoflaw',
                          '2000vanda', '2005vanda', '2010vanda']]

indexes.columns = ['ruleoflaw2000', 'ruleoflaw2005', 'ruleoflaw2010',  'vanda2000', 'vanda2005', 'vanda2010']

#standardized scores
indexes = indexes.apply(lambda x: (x-x.mean())/x.std())

indexes['vandasquared2000'] = indexes['vanda2000']**2
indexes['vandasquared2005'] = indexes['vanda2005']**2
indexes['vandasquared2010'] = indexes['vanda2010']**2




In [16]:
#Construct female years of schooling data
femaleschooling = pd.read_csv("2013F2599.csv")
femaleschooling = femaleschooling.fillna(method='ffill')
femaleschooling['Year'] = femaleschooling['Year'].apply(lambda x: int(x))
femaleschooling = femaleschooling.dropna()
femaleschooling = femaleschooling[['Country', 'Year', 'Total Schooling']]
femaleschooling = femaleschooling.rename(columns = {'Country': 'country', 'Year': 'year', 'Total Schooling': 'femaleschooling'})
femaleschooling = femaleschooling[femaleschooling['year'].isin([2000, 2005, 2010])]
femaleschooling = femaleschooling.drop_duplicates()

In [17]:
#construct panel data in wide format

paneldata = pd.concat([GDPgrowth, TFPgrowth, logGDP, logTFP, yearsofschooling, logfertilityrate, 
                      loglifeexpectancy, gcratio, inflation,  indexes], axis = 1)
paneldata['country'] = merged8['country']
paneldata.columns
paneldata

,gdpgrowth2000,gdpgrowth2005,gdpgrowth2010,tfpgrowth2000,tfpgrowth2005,tfpgrowth2010,gdp2000,gdp2005,gdp2010,tfp2000,...,ruleoflaw2000,ruleoflaw2005,ruleoflaw2010,vanda2000,vanda2005,vanda2010,vandasquared2000,vandasquared2005,vandasquared2010,country
0,0.104004,0.320711,0.129353,0.010692,0.105593,-0.014859,13.175429,13.274373,13.552543,-0.139970,...,-0.391472,-0.776114,-0.829332,0.184178,0.007864,0.177265,0.033922,0.000062,0.031423,Argentina
1,0.808074,0.208409,0.200263,0.900307,0.006927,0.144122,9.304577,9.896839,10.086143,-0.691459,...,-0.718949,-0.623014,-0.728167,-0.650817,-0.910273,-1.094504,0.423563,0.828596,1.197939,Armenia
2,0.185403,0.142043,0.120231,0.065458,-0.001489,0.003269,13.515164,13.685247,13.818065,-0.064812,...,1.503107,1.483717,1.509882,1.292980,1.328452,1.290049,1.671796,1.764785,1.664227,Australia
3,0.088989,0.066370,0.042867,0.011002,-0.000589,-0.008203,12.585997,12.671246,12.735506,-0.015605,...,1.593749,1.617317,1.544972,1.131080,1.191468,1.301119,1.279342,1.419596,1.692910,Austria
4,0.286057,0.310533,0.164899,0.032611,-0.158898,0.044805,10.225749,10.477330,10.747764,0.153109,...,0.055317,0.374997,0.189613,-1.369287,-1.072235,-1.218978,1.874946,1.149687,1.485909,Bahrain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,0.249368,0.170011,0.191171,0.096580,-0.046420,-0.021970,13.627418,13.850056,14.007069,-0.061672,...,-0.253834,-0.067769,-0.133010,-0.560893,-0.279120,-0.292332,0.314601,0.077908,0.085458,Turkey
90,0.446740,0.051385,-0.015625,0.450243,0.094592,0.006738,12.587884,12.957197,13.007306,-0.507876,...,-1.346951,-0.998447,-1.044964,-0.899602,-0.583772,-0.291473,0.809284,0.340789,0.084957,Ukraine
91,0.148855,0.019776,0.085023,0.069728,-0.028972,0.014602,14.430162,14.568928,14.588511,-0.048405,...,1.447955,1.330049,1.509024,1.137093,1.261134,1.153568,1.292981,1.590459,1.330718,United Kingdom
92,0.009168,0.335807,0.181938,-0.001198,0.171046,0.057874,10.623824,10.632950,10.922485,-0.157832,...,0.346287,0.241010,0.490599,0.785951,0.665591,1.002702,0.617719,0.443012,1.005411,Uruguay


In [18]:
#Construct panel data in long format
paneldata_long = pd.DataFrame()

for i in np.arange(0, paneldata.shape[1]-1, step = 3):
    paneldata_long[paneldata.columns[i][:-4]] = paneldata[[paneldata.columns[i], 
                                                    paneldata.columns[i+1], paneldata.columns[i+2]]].to_numpy().flatten()

paneldata_long['country'] = np.repeat(paneldata['country'].to_numpy(), 3)
year = np.tile([2000, 2005, 2010], paneldata.shape[0])
paneldata_long['year'] = year
paneldata_long['inflation'] = paneldata_long['inflation'] *0.01
paneldata_long['gcratio'] = paneldata_long['gcratio']  *0.01
paneldata_long['femaleyoung'] = paneldata_long['femaleyoung']  *0.01
paneldata_long = pd.merge(paneldata_long, femaleschooling, how = 'left')
paneldata_long.to_csv("panelregressioninputs.csv", index = False)

In [19]:
paneldata_long.columns

Index(['gdpgrowth', 'tfpgrowth', 'gdp', 'tfp', 'sexratio', 'maleyoung',
       'femaleyoung', 'totalyoung', 'logfertilityrate', 'loglifeexpectancy',
       'gcratio', 'inflation', 'ruleoflaw', 'vanda', 'vandasquared', 'country',
       'year', 'femaleschooling'],
      dtype='object')

In [20]:
#Initial level of education quality
eduquality = pd.read_csv('growthdata.csv')
eduquality = eduquality.drop_duplicates(keep = 'last', subset = ['country'])
eduquality = eduquality[['country', '2000eduquality', '2006eduquality']]
eduquality.columns = ['country', 'eduquality2000', 'eduquality2006']
eduquality
paneldata_eduqua =  pd.merge(paneldata, eduquality, how = 'inner', on = 'country')
#standardized scores
paneldata_eduqua[['eduquality2000', 'eduquality2006']] = paneldata_eduqua[['eduquality2000', 'eduquality2006']].apply(lambda x: (x-x.mean())/x.std())

In [21]:
newcolumns = [x for x in list(paneldata_eduqua.columns) if  "2010" not in x and 'country' not in x]
print(newcolumns)
newcolumns.append('country')
print(newcolumns)
paneldata_eduqua = paneldata_eduqua[newcolumns]
paneldata_eduqua

['gdpgrowth2000', 'gdpgrowth2005', 'tfpgrowth2000', 'tfpgrowth2005', 'gdp2000', 'gdp2005', 'tfp2000', 'tfp2005', 'sexratio2000', 'sexratio2005', 'maleyoung2000', 'maleyoung2005', 'femaleyoung2000', 'femaleyoung2005', 'totalyoung2000', 'totalyoung2005', 'logfertilityrate2000', 'logfertilityrate2005', 'loglifeexpectancy2000', 'loglifeexpectancy2005', 'gcratio2000', 'gcratio2005', 'inflation2000', 'inflation2005', 'ruleoflaw2000', 'ruleoflaw2005', 'vanda2000', 'vanda2005', 'vandasquared2000', 'vandasquared2005', 'eduquality2000', 'eduquality2006']
['gdpgrowth2000', 'gdpgrowth2005', 'tfpgrowth2000', 'tfpgrowth2005', 'gdp2000', 'gdp2005', 'tfp2000', 'tfp2005', 'sexratio2000', 'sexratio2005', 'maleyoung2000', 'maleyoung2005', 'femaleyoung2000', 'femaleyoung2005', 'totalyoung2000', 'totalyoung2005', 'logfertilityrate2000', 'logfertilityrate2005', 'loglifeexpectancy2000', 'loglifeexpectancy2005', 'gcratio2000', 'gcratio2005', 'inflation2000', 'inflation2005', 'ruleoflaw2000', 'ruleoflaw2005', 

,gdpgrowth2000,gdpgrowth2005,tfpgrowth2000,tfpgrowth2005,gdp2000,gdp2005,tfp2000,tfp2005,sexratio2000,sexratio2005,...,inflation2005,ruleoflaw2000,ruleoflaw2005,vanda2000,vanda2005,vandasquared2000,vandasquared2005,eduquality2000,eduquality2006,country
0,0.104004,0.320711,0.010692,0.105593,13.175429,13.274373,-0.139970,-0.129335,1.077512,1.080142,...,8.847745,-0.391472,-0.776114,0.184178,0.007864,0.033922,0.000062,-0.988346,-1.374981,Argentina
1,0.808074,0.208409,0.900307,0.006927,9.304577,9.896839,-0.691459,-0.049444,1.211430,1.244295,...,4.059756,-0.718949,-0.623014,-0.650817,-0.910273,0.423563,0.828596,-0.536347,-0.149016,Armenia
2,0.185403,0.142043,0.065458,-0.001489,13.515164,13.685247,-0.064812,-0.001407,1.078021,1.073567,...,2.942467,1.503107,1.483717,1.292980,1.328452,1.671796,1.764785,1.011418,0.965386,Australia
3,0.088989,0.066370,0.011002,-0.000589,12.585997,12.671246,-0.015605,-0.004663,1.116425,1.113075,...,1.929043,1.593749,1.617317,1.131080,1.191468,1.279342,1.419596,0.590778,0.721918,Austria
4,0.286057,0.310533,0.032611,-0.158898,10.225749,10.477330,0.153109,0.185199,0.798332,0.722536,...,2.834372,0.055317,0.374997,-1.369287,-1.072235,1.874946,1.149687,-0.959533,-0.513783,Bahrain
5,0.094037,0.071456,0.003059,-0.028577,12.770457,12.860331,0.034283,0.037337,1.091491,1.085111,...,2.167454,1.093275,1.031422,1.182621,1.236389,1.398592,1.528657,0.832129,0.972833,Belgium
6,0.194712,0.249800,-0.126396,-0.087865,9.778650,9.956555,0.240368,0.105240,1.042240,1.039997,...,9.595186,0.350530,0.408436,0.453681,0.354996,0.205827,0.126022,-2.295125,-2.470992,Botswana
7,0.151814,0.243000,-0.012619,0.058511,14.508261,14.649599,-0.030510,-0.043210,1.062245,1.066929,...,5.047513,-0.449792,-0.652007,0.061233,0.208912,0.003749,0.043644,-1.733085,-1.573124,Brazil
8,0.327013,0.163770,0.000955,-0.096581,11.168376,11.451307,0.095337,0.096292,1.107269,1.116747,...,7.160978,-0.362657,-0.320929,0.228565,0.355300,0.052242,0.126238,-0.409104,-0.831551,Bulgaria
9,0.133707,0.065183,0.001180,-0.040477,13.973266,14.098759,0.035101,0.036280,1.069448,1.069207,...,1.804740,1.446291,1.421456,1.356857,1.328264,1.841061,1.764286,1.011418,1.085219,Canada


In [22]:
paneldata_eduqua_long = pd.DataFrame()

for i in np.arange(0, paneldata_eduqua.shape[1]-1, step = 2):
    paneldata_eduqua_long[paneldata_eduqua.columns[i][:-4]] = paneldata_eduqua[[paneldata_eduqua.columns[i], 
                                                    paneldata_eduqua.columns[i+1]]].to_numpy().flatten()

paneldata_eduqua_long['country'] = np.repeat(paneldata_eduqua['country'].to_numpy(), 2)
year = np.tile([2000, 2005], paneldata_eduqua.shape[0])
paneldata_eduqua_long['year'] = year

In [23]:
paneldata_eduqua_long.head(20)
paneldata_eduqua_long['inflation'] = paneldata_eduqua_long['inflation'] *0.01
paneldata_eduqua_long['gcratio'] = paneldata_eduqua_long['gcratio']  *0.01
paneldata_eduqua_long['femaleyoung'] = paneldata_eduqua_long['femaleyoung']  *0.01
paneldata_eduqua_long = pd.merge(paneldata_eduqua_long, femaleschooling, how = 'left')
paneldata_eduqua_long.to_csv("panelregressioninputs_eduqua.csv", index = False)